In [1]:
# Goal: Topic models with NMF (Gensim, Scikit-learn), visualizaion (preliminary)
# Result: implementations of topic models with high (coherence scores, other measure?) from each library
# Output: (wordcloud, t-SNE?)
# TO DO: save corpus, dictionary for reuse

In [2]:
# Scikit-learn
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel